<a href="https://colab.research.google.com/github/Zhiyuan-03/AI_in_Transportation_Exercise/blob/main/student-t.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Load data
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
# Step 1: Upload the file
uploaded = files.upload()

# Step 2: Load the uploaded CSV file
# Replace 'cf_pair25_v1.csv' with the exact filename you uploaded
data = pd.read_csv('cf_pair25_v1.csv')

Saving cf_pair25_v1.csv to cf_pair25_v1.csv


In [3]:
# ===============================
# 0. Imports and configuration
# ===============================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import least_squares, minimize
from sklearn.metrics import r2_score
from scipy.stats import anderson, shapiro
from scipy.special import gammaln
import scipy.stats as stats

# -------------------------------
# Config
# -------------------------------
TIME_STEP = 0.1    # seconds
SIGMA      = 1.0   # noise std for Gaussian LL (used in non-Student-t codes)
MOP        = "speed"  # "speed" or "position"
PLOT_DIR   = "Plots"
os.makedirs(PLOT_DIR, exist_ok=True)

# ===============================
# 1. IDM model
# ===============================
def idm_model(s, v, delta_v, params, T=1.5, v0=30, s0=1):
    a, b = params
    delta = 4
    # s_star as in classical IDM
    s_star = s0 + v * T + (v * delta_v) / (2 * np.sqrt(a * b))
    acceleration = a * (1 - (v / v0) ** delta - (s_star / s) ** 2)
    return acceleration

# ===============================
# 2. Calibration function
# ===============================
def calibration(params, df, mop="position", time_step=TIME_STEP, sigma=SIGMA):
    """
    Simulate each vehicle's trajectory using `params` and return:
    residuals, mean_gof, negative_log_likelihood, mean_r2 (pos, speed), trajectories, speed_curves
    Note: negative_log_likelihood uses Gaussian LL with `sigma` when mop is position/speed.
    """
    residuals = []
    total_gof = 0.0
    total_ll  = 0.0
    r2_list   = []
    vehicle_count = 0
    trajectories = {}
    speed_curves = {}

    grouped = df.groupby('Vehicle_ID_f')
    for vehicle_id, data in grouped:
        v0 = data["v_f"].iloc[0]
        x0 = data["x_f"].iloc[0]
        v_lead = data["v_l"].values
        x_lead = data["x_l"].values
        obs_v = data["v_f"].values
        obs_x = data["x_f"].values
        sim_v = [v0]
        sim_x = [x0]
        time     = data["Frame_ID_f"].values
        length_l = data['length_l'].values
        length_f = data['length_f'].values

        for i in range(1, len(data)):
            vL = v_lead[i-1]; xL = x_lead[i-1]
            v  = sim_v[i-1]; x  = sim_x[i-1]
            s = xL - x - (length_l[i-1] + length_f[i-1]) / 2.0
            delta_v = v - vL
            acc = idm_model(s, v, delta_v, params)
            v_next = max(v + acc * time_step, 0.0)
            x_next = x + (v_next + v) / 2.0 * time_step
            sim_v.append(v_next); sim_x.append(x_next)

        if mop == "position":
            res = obs_x - np.array(sim_x)
            gof = np.mean(res**2)
            ll  = -0.5 * np.sum((res / sigma) ** 2 + np.log(2 * np.pi * sigma**2))
        elif mop == "speed":
            res = obs_v - np.array(sim_v)
            gof = np.mean(res**2)
            ll  = -0.5 * np.sum((res / sigma) ** 2 + np.log(2 * np.pi * sigma**2))
        else:
            raise ValueError("mop must be 'position' or 'speed'.")

        residuals.extend(res)
        total_gof += gof; total_ll += ll; vehicle_count += 1
        # compute R^2 (catch degenerate cases)
        try:
            r2_pos   = r2_score(obs_x, sim_x)
        except Exception:
            r2_pos = np.nan
        try:
            r2_speed = r2_score(obs_v, sim_v)
        except Exception:
            r2_speed = np.nan
        r2_list.append([r2_pos, r2_speed])

        trajectories[vehicle_id] = {"obs_x": obs_x, "sim_x": np.array(sim_x), "x_lead": x_lead, "time": time}
        speed_curves[vehicle_id] = {"obs_v": obs_v, "sim_v": np.array(sim_v), "v_lead": v_lead, "time": time}

    mean_r2 = np.nanmean(np.array(r2_list), axis=0)
    return residuals, total_gof / max(vehicle_count, 1), -total_ll, mean_r2, trajectories, speed_curves

def calibration_gof(params, data, mop, number):
    """
    Helper wrapper used during optimization.
    If number == 0 -> returns full residual vector (for least_squares)
    If number == 1 -> returns GOF scalar (used with minimize to minimize GOF)
    If number == 2 -> returns negative log-likelihood scalar (used with minimize)
    """
    result = calibration(params, data, mop)
    return result[number]

# ===============================
# 3. NLL functions
# ===============================
def simulate_v(data, params, time_step=TIME_STEP):
    v_f = data['v_f'].iloc[0]; x_f = data['x_f'].iloc[0]
    v_l = data['v_l'].values; x_l = data['x_l'].values
    v_sim = [v_f]; x_sim = [x_f]
    lengths_l = data['length_l'].values; lengths_f = data['length_f'].values
    for i in range(1, len(data)):
        s = x_l[i-1] - x_sim[-1] - (lengths_l[i-1] + lengths_f[i-1]) / 2.0
        delta_v = v_sim[-1] - v_l[i-1]
        acc = idm_model(s, v_sim[-1], delta_v, params)
        new_v = max(v_sim[-1] + acc * time_step, 0.0)
        new_x = x_sim[-1] + (new_v + v_sim[-1]) / 2.0 * time_step
        v_sim.append(new_v); x_sim.append(new_x)
    return np.array(v_sim)

def log_likelihood(params, df, time_step=TIME_STEP, sigma=SIGMA):
    """
    Gaussian negative log-likelihood across vehicles (returns -total_ll, suitable for minimization).
    """
    total_ll = 0.0
    for vid, data in df.groupby('Vehicle_ID_f'):
        v_obs = data['v_f'].values
        v_sim = simulate_v(data, params, time_step)
        residuals = v_obs - v_sim
        ll = -0.5 * np.sum((residuals / sigma) ** 2 + np.log(2 * np.pi * sigma**2))
        total_ll += ll
    return -total_ll

def nll_student_t(params, df, time_step=TIME_STEP):
    """
    Negative log-likelihood for Student-t errors.
    params = [a, b, sigma, nu]
    We return -total_log_likelihood (so minimizing this finds MLE).
    """
    a, b, sigma, nu = params
    if sigma <= 0 or nu <= 0:
        return 1e12  # penalty for invalid parameters
    total_ll = 0.0
    c = gammaln((nu+1)/2) - gammaln(nu/2) - 0.5*np.log(nu*np.pi) - np.log(sigma)
    for vid, data in df.groupby('Vehicle_ID_f'):
        v_obs = data['v_f'].values
        v_sim = simulate_v(data, [a, b], time_step)
        r = v_obs - v_sim
        # Student-t log pdf (vectorized)
        ll = np.sum(c - ((nu+1)/2)*np.log(1 + (r**2)/(nu*sigma**2)))
        total_ll += ll
    return -total_ll

# ===============================
# 4. Data loading and split
# ===============================
data = pd.read_csv('cf_pair25_v1.csv')
vehicle_ids = data["Vehicle_ID_f"].unique()
N = len(vehicle_ids)
vehicle_ids_eval      = vehicle_ids[-5:]
vehicle_ids_crossval  = vehicle_ids[:-5]
print("Evaluation vehicle IDs:", [int(x) for x in vehicle_ids_eval])

# ===============================
# 5. Cross-validation
# ===============================
mop = MOP
trajectories = {}; speed_curves = {}
all_params_1, all_params_2, all_params_3, all_params_4 = [], [], [], []
test_gof = {'least_squares': [], 'minimize_gof': [], 'minimize_ll': [], 'minimize_student_t': []}

bounds_1 = ([0.1, 0.1], [7.0, 7.0])  # for least_squares (a,b)
bounds_2 = [(0.1, 7.0), (0.1, 7.0)]  # for minimize with (a,b)
bounds_4 = [(0.1, 7.0), (0.1, 7.0), (1e-3, 5.0), (2.1, 200.0)]  # for (a,b,sigma,nu)
initial_guess = [0.7, 1.5]

# We'll iterate through cross-validation folds
for start in range(0, len(vehicle_ids_crossval), 4):
    test_ids  = vehicle_ids[start:start+4]
    train_ids = np.setdiff1d(vehicle_ids, test_ids)
    train_df = data[data["Vehicle_ID_f"].isin(train_ids)]
    test_df  = data[data["Vehicle_ID_f"].isin(test_ids)]

    # ----- result_1: least_squares on residual vector (minimize residuals directly) -----
    try:
        result_1 = least_squares(calibration_gof, x0=initial_guess, args=(train_df, mop, 0),
                                 bounds=bounds_1)
        params_1 = result_1.x
    except Exception as e:
        print("least_squares failed in fold starting at", start, ":", e)
        params_1 = np.array(initial_guess)
    all_params_1.append(params_1)

    # ----- result_2: minimize GOF (mean squared error) -----
    try:
        result_2 = minimize(calibration_gof, x0=initial_guess, args=(train_df, mop, 1),
                            bounds=bounds_2, method="L-BFGS-B")
        params_2 = result_2.x
    except Exception as e:
        print("minimize (gof) failed in fold starting at", start, ":", e)
        params_2 = np.array(initial_guess)
    all_params_2.append(params_2)

    # ----- result_3: minimize Gaussian negative log-likelihood (using calibration) -----
    try:
        result_3 = minimize(calibration_gof, x0=initial_guess, args=(train_df, mop, 2),
                            bounds=bounds_2, method="L-BFGS-B")
        params_3 = result_3.x
    except Exception as e:
        print("minimize (gaussian ll) failed in fold starting at", start, ":", e)
        params_3 = np.array(initial_guess)
    all_params_3.append(params_3)

    # ----- result_4: minimize Student-t negative log-likelihood (a, b, sigma, nu) -----
    try:
        # initial for student-t: (a,b,sigma,nu)
        init_student = [0.7, 1.5, 1.0, 5.0]
        result_4 = minimize(nll_student_t, x0=init_student, args=(train_df,),
                            bounds=bounds_4, method="L-BFGS-B",
                            options={'maxiter': 2000})
        params_4 = result_4.x
    except Exception as e:
        print("minimize (student-t) failed in fold starting at", start, ":", e)
        params_4 = np.array([0.7, 1.5, 1.0, 5.0])
    all_params_4.append(params_4)

    # ----- Evaluate each model on the test set using calibration (GOF) -----
    # least_squares params
    try:
        _, gof1, negll1, r2_1, _, _ = calibration(params_1, test_df, mop=mop)
    except Exception as e:
        gof1 = np.nan; negll1 = np.nan; r2_1 = [np.nan, np.nan]
    # minimize GOF params
    try:
        _, gof2, negll2, r2_2, _, _ = calibration(params_2, test_df, mop=mop)
    except Exception as e:
        gof2 = np.nan; negll2 = np.nan; r2_2 = [np.nan, np.nan]
    # minimize Gaussian LL params
    try:
        _, gof3, negll3, r2_3, _, _ = calibration(params_3, test_df, mop=mop)
    except Exception as e:
        gof3 = np.nan; negll3 = np.nan; r2_3 = [np.nan, np.nan]
    # student-t params (note params_4 contains sigma and nu too)
    try:
        _, gof4, negll4, r2_4, _, _ = calibration(params_4[:2], test_df, mop=mop)
    except Exception as e:
        gof4 = np.nan; negll4 = np.nan; r2_4 = [np.nan, np.nan]

    test_gof['least_squares'].append(gof1)
    test_gof['minimize_gof'].append(gof2)
    test_gof['minimize_ll'].append(gof3)
    test_gof['minimize_student_t'].append(gof4)

    print(f"Fold starting {start}: params_1={params_1}, params_2={params_2}, params_3={params_3}, params_4={params_4}")
    print(f"  test GOF (MSE) -> LS: {gof1:.4f}, GOF-min: {gof2:.4f}, Gauss-LL: {gof3:.4f}, Student-t: {gof4:.4f}")

# ===============================
# 6. Summaries across folds
# ===============================
def summarise_params(param_list, name):
    mat = np.array(param_list)
    print(f"\n{name} summary over folds:")
    if mat.size == 0:
        print("  (no results)")
        return
    for i in range(mat.shape[1]):
        col = mat[:, i]
        print(f"  param[{i}]: mean={np.nanmean(col):.4f}, std={np.nanstd(col):.4f}")

summarise_params(all_params_1, "least_squares (a,b)")
summarise_params(all_params_2, "minimize_gof (a,b)")
summarise_params(all_params_3, "minimize_ll (a,b)")
summarise_params(all_params_4, "minimize_student_t (a,b,sigma,nu)")

print("\nMean test GOFs (MSE) across folds:")
for k, v in test_gof.items():
    print(f"  {k}: {np.nanmean(v):.6f} (n={len(v)})")

# ===============================
# 7. Final Student-t fit on full dataset & evaluation
# ===============================
print("\nFitting final Student-t model on the full dataset...")
init_student = [0.7, 1.5, 1.0, 5.0]
try:
    final_student = minimize(nll_student_t, x0=init_student, args=(data,),
                             bounds=bounds_4, method="L-BFGS-B",
                             options={'maxiter': 4000})
    a_hat, b_hat, sigma_hat, nu_hat = final_student.x
    final_negll = float(final_student.fun)
    final_ll = -final_negll
    print(f"Final student-t params: a={a_hat:.4f}, b={b_hat:.4f}, sigma={sigma_hat:.4f}, nu={nu_hat:.4f}")
    print(f"Final log-likelihood: {final_ll:.4f} (negll={final_negll:.4f})")
except Exception as e:
    print("Final student-t fit failed:", e)
    a_hat, b_hat, sigma_hat, nu_hat = (0.7, 1.5, 1.0, 5.0)
    final_negll = np.nan
    final_ll = np.nan

# Compute AIC / BIC
k_params = 4
n_obs = len(data)  # total number of rows (observations)
if not np.isnan(final_negll):
    AIC = 2 * k_params + 2 * final_negll
    BIC = k_params * np.log(n_obs) + 2 * final_negll
    print(f"AIC = {AIC:.3f}, BIC = {BIC:.3f}, n_obs = {n_obs}")
else:
    print("AIC/BIC cannot be computed because final_negll is NaN")

# ===============================
# 8. Residual analysis for Student-t model (final fit)
# ===============================
# Collect residuals across all vehicles
residuals_all = []
std_residuals_all = []
index_all = []
vehicle_ix = []
for vid, group in data.groupby('Vehicle_ID_f'):
    v_obs = group['v_f'].values
    v_sim = simulate_v(group, [a_hat, b_hat], TIME_STEP)
    r = v_obs - v_sim
    residuals_all.extend(r)
    std_residuals_all.extend(r / sigma_hat)
    # index: use order within vehicle relative to dataset-global index for plotting
    index_all.extend(list(group.index.values))
    vehicle_ix.extend([vid] * len(r))

residuals_all = np.array(residuals_all)
std_residuals_all = np.array(std_residuals_all)
index_all = np.array(index_all)

# Summary statistics
print("\nResidual summary (Student-t fit):")
print(f"  n_res = {len(residuals_all)}")
print(f"  residual mean = {np.mean(residuals_all):.6f}, std = {np.std(residuals_all):.6f}")
print(f"  standardized residual mean = {np.mean(std_residuals_all):.6f}, std = {np.std(std_residuals_all):.6f}")

# Normality tests on standardized residuals (note: these assume normality; for student-t one expects t tails)
try:
    sh_w, sh_p = shapiro(std_residuals_all if len(std_residuals_all) <= 5000 else std_residuals_all[:5000])
    print(f"Shapiro-Wilk on standardized residuals (first 5000 if >5000): W={sh_w:.4f}, p={sh_p:.4g}")
except Exception as e:
    print("Shapiro test failed:", e)

try:
    ad_res = anderson(std_residuals_all)
    print("Anderson-Darling statistic:", ad_res.statistic, "critical:", ad_res.critical_values)
except Exception as e:
    print("Anderson test failed:", e)

# -------------------------
# QQ plot vs Student-t
# -------------------------
plt.figure(figsize=(6,6))
# Use scipy.stats.probplot with distribution = t and sparams=(nu,)
# probplot will produce points against theoretical quantiles of Student-t (which has mean 0)
try:
    # If nu_hat is not sensible, fallback
    nu_for_qq = max(nu_hat, 2.1)
    (osm, osr), (slope, intercept, r) = stats.probplot(std_residuals_all, dist=stats.t, sparams=(nu_for_qq,), fit=True)
    # probplot with dist=t returns arrays suitable for scatterplot, but to draw line manually:
    plt.scatter(osm, osr, marker='o', s=8, alpha=0.6, label='Std residuals')
    # Draw fitted line
    x_line = np.array([np.min(osm), np.max(osm)])
    y_line = intercept + slope * x_line
    plt.plot(x_line, y_line, color='k', lw=1.0, label='Fit line')
    plt.xlabel('Theoretical quantiles (Student-t)')
    plt.ylabel('Ordered standardized residuals')
    plt.title(f'Q–Q plot (Student-t dist, df={nu_for_qq:.2f})')
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.4)
    plt.tight_layout()
    qq_path = os.path.join(PLOT_DIR, 'qq_student_t.png')
    plt.savefig(qq_path, dpi=200)
    plt.close()
    print("Saved Q–Q plot vs Student-t to", qq_path)
except Exception as e:
    print("QQ-plot failed:", e)

# -------------------------
# Histogram of standardized residuals
# -------------------------
plt.figure(figsize=(6,4))
plt.hist(std_residuals_all, bins=50, density=True, alpha=0.7)
# overlay Student-t pdf for comparison
try:
    xs = np.linspace(np.percentile(std_residuals_all, 0.5), np.percentile(std_residuals_all, 99.5), 300)
    pdf_t = stats.t.pdf(xs, df=nu_hat)
    plt.plot(xs, pdf_t, lw=2, label=f"Student-t pdf (df={nu_hat:.2f})")
except Exception:
    pass
plt.xlabel('Standardized residuals (r / sigma_hat)')
plt.ylabel('Density')
plt.title('Histogram of standardized residuals (Student-t fit)')
plt.grid(True, linestyle=':', alpha=0.4)
plt.legend()
hist_path = os.path.join(PLOT_DIR, 'hist_std_residuals.png')
plt.tight_layout()
plt.savefig(hist_path, dpi=200)
plt.close()
print("Saved histogram of standardized residuals to", hist_path)

# -------------------------
# Residual vs. index plot
# -------------------------
plt.figure(figsize=(10,4))
plt.scatter(index_all, residuals_all, s=8, alpha=0.6)
plt.axhline(0, color='k', lw=1)
plt.xlabel('Data index (global row index)')
plt.ylabel('Residual (v_obs - v_sim)')
plt.title('Residuals vs. Index (order in dataset)')
plt.grid(True, linestyle=':', alpha=0.4)
res_index_path = os.path.join(PLOT_DIR, 'residuals_index.png')
plt.tight_layout()
plt.savefig(res_index_path, dpi=200)
plt.close()
print("Saved residuals vs index plot to", res_index_path)

# -------------------------
# Residuals by vehicle (optional quick check) - save one plot per first few vehicles
# -------------------------
unique_vehicles = np.unique(vehicle_ix)
for v in unique_vehicles[:6]:
    mask = np.array(vehicle_ix) == v
    plt.figure(figsize=(8,3))
    plt.plot(np.where(mask)[0], residuals_all[mask], marker='o', linestyle='-', markersize=4)
    plt.axhline(0, color='k', lw=1)
    plt.xlabel('Observation index within dataset')
    plt.ylabel('Residual')
    plt.title(f'Residuals for Vehicle {v}')
    plt.grid(True, linestyle=':', alpha=0.4)
    fn = os.path.join(PLOT_DIR, f'res_vehicle_{int(v)}.png')
    plt.tight_layout(); plt.savefig(fn, dpi=180); plt.close()

print(f"Saved residual-by-vehicle plots for up to 6 vehicles in {PLOT_DIR}")

# ===============================
# 9. Final evaluation on held-out evaluation vehicles
# ===============================
eval_df = data[data["Vehicle_ID_f"].isin(vehicle_ids_eval)]
_, eval_gof, eval_negll, eval_r2, eval_trajectories, eval_speedcurves = calibration([a_hat, b_hat], eval_df, mop=mop)
print("\nFinal evaluation on held-out evaluation vehicles:")
print(f"  MSE (GOF) = {eval_gof:.6f}, negative-LL (Gaussian, using sigma={SIGMA}) = {eval_negll:.6f}")
print(f"  Mean R^2 (pos, speed) = {eval_r2}")

# Optionally visualize observed vs simulated for a held-out vehicle
for vid in vehicle_ids_eval:
    g = eval_trajectories.get(vid, None)
    sc = eval_speedcurves.get(vid, None)
    if g is None or sc is None:
        continue
    t = g['time']
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(t, g['obs_x'], label='obs_x')
    plt.plot(t, g['sim_x'], label='sim_x')
    plt.xlabel('Frame ID'); plt.ylabel('Position'); plt.title(f'Vehicle {int(vid)} position')
    plt.legend(); plt.grid(True, linestyle=':', alpha=0.4)
    plt.subplot(1,2,2)
    plt.plot(t, sc['obs_v'], label='obs_v')
    plt.plot(t, sc['sim_v'], label='sim_v')
    plt.xlabel('Frame ID'); plt.ylabel('Speed'); plt.title(f'Vehicle {int(vid)} speed')
    plt.legend(); plt.grid(True, linestyle=':', alpha=0.4)
    fn = os.path.join(PLOT_DIR, f'eval_vehicle_{int(vid)}_obs_vs_sim.png')
    plt.tight_layout(); plt.savefig(fn, dpi=180); plt.close()

print(f"Saved evaluation obs-vs-sim plots for held-out vehicles to {PLOT_DIR}")

# ===============================
# End of script
# ===============================


Evaluation vehicle IDs: [12638, 10, 9683, 3054, 5167]
Fold starting 0: params_1=[0.75593846 1.38109498], params_2=[0.73842678 1.32395696], params_3=[0.75590044 1.38093823], params_4=[0.85660951 1.79340337 0.27126915 8.35425239]
  test GOF (MSE) -> LS: 0.2197, GOF-min: 0.2191, Gauss-LL: 0.2197, Student-t: 0.2215
Fold starting 4: params_1=[0.71862851 1.81564563], params_2=[0.70583855 1.84258958], params_3=[0.71865855 1.81582272], params_4=[0.86515607 2.32532842 0.27841002 5.73665023]
  test GOF (MSE) -> LS: 0.0833, GOF-min: 0.0842, Gauss-LL: 0.0833, Student-t: 0.0838
Fold starting 8: params_1=[0.66436438 1.39249402], params_2=[0.64804878 1.38280044], params_3=[0.66433324 1.39233505], params_4=[0.84497041 1.94062609 0.27366904 5.16285435]
  test GOF (MSE) -> LS: 0.0761, GOF-min: 0.0762, Gauss-LL: 0.0761, Student-t: 0.0756
Fold starting 12: params_1=[0.67615806 1.45376773], params_2=[0.66204301 1.44916326], params_3=[0.67610148 1.45345953], params_4=[0.83148178 1.91480584 0.26648982 4.9855

In [5]:
!ls Plots
from IPython.display import Image
Image("Plots/hist_std_residuals.png")
from google.colab import files
#files.download("Plots/qq_student_t.png")
files.download("Plots/hist_std_residuals.png")

eval_vehicle_10_obs_vs_sim.png	   hist_std_residuals.png  res_vehicle_15.png
eval_vehicle_12638_obs_vs_sim.png  qq_student_t.png	   res_vehicle_1939.png
eval_vehicle_3054_obs_vs_sim.png   residuals_index.png	   res_vehicle_26.png
eval_vehicle_5167_obs_vs_sim.png   res_vehicle_10.png	   res_vehicle_9.png
eval_vehicle_9683_obs_vs_sim.png   res_vehicle_1342.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
import numpy as np
from scipy.stats import norm

def compute_normal_mle_aic_bic(residuals):
    """
    Computes log-likelihood, AIC, BIC assuming a Gaussian model with
    mean zero and sigma estimated by MLE (population std).
    """

    n = len(residuals)

    # MLE sigma uses ddof=0 (population)
    sigma_mle = np.std(residuals, ddof=0)

    # Compute Gaussian log-likelihood
    ll = np.sum(norm.logpdf(residuals, loc=0.0, scale=sigma_mle))

    # Number of parameters:
    # a, b, and σ  -->  k = 3
    k = 3

    # AIC & BIC
    aic = 2 * k - 2 * ll
    bic = k * np.log(n) - 2 * ll

    return ll, aic, bic, sigma_mle


# ---------------------------
# Example usage inside your script:
# ---------------------------

ll_norm, aic_norm, bic_norm, sigma_norm = compute_normal_mle_aic_bic(residuals_all)

print(f"\nGaussian MLE σ = {sigma_norm:.6f}")
print(f"Gaussian log-likelihood = {ll_norm:.4f}")
print(f"Gaussian AIC = {aic_norm:.4f}")
print(f"Gaussian BIC = {bic_norm:.4f}\n")



Gaussian MLE σ = 0.336451
Gaussian log-likelihood = -5770.8060
Gaussian AIC = 11547.6120
Gaussian BIC = 11570.8809

